In [9]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [10]:
train_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/train'
val_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/val'
test_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/test'

# Defining Model

In [11]:
import keras
from keras import layers
from keras import models

In [12]:
std_model = models.Sequential([
    layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (299,299,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation = 'relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(10, activation = 'softmax')
    
])

std_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 297, 297, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 148, 148, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 146, 146, 32)      4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 170528)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                10913856  
_________________________________________________________________
dense_4 (Dense)              (None, 10)               

# Model

In [13]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical')

Found 1233 images belonging to 10 classes.
Found 200 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


In [14]:
from keras.losses import categorical_crossentropy as logloss
from keras import backend as K

def loss(y_true, y_pred):
    entropy = -K.mean(K.sum(y_pred*K.log(y_pred), 1))
    beta = 0.1
    return logloss(y_true, y_pred) - beta*entropy

In [15]:
from keras import optimizers

std_model.compile(loss=loss,
    optimizer=optimizers.SGD(lr=1e-2),
    metrics=['categorical_crossentropy', 'accuracy'])

In [16]:
history = std_model.fit_generator(
    train_generator,
    steps_per_epoch=80,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=20)

Epoch 1/10
80/80 [==============================] - 12s 153ms/step - loss: 1.9712 - categorical_crossentropy: 2.1923 - accuracy: 0.2517 - val_loss: 2.0786 - val_categorical_crossentropy: 2.3580 - val_accuracy: 0.1700
Epoch 2/10
80/80 [==============================] - 12s 150ms/step - loss: 1.7161 - categorical_crossentropy: 1.9188 - accuracy: 0.3553 - val_loss: 1.7364 - val_categorical_crossentropy: 1.8364 - val_accuracy: 0.4000
Epoch 3/10
80/80 [==============================] - 14s 171ms/step - loss: 1.5061 - categorical_crossentropy: 1.6893 - accuracy: 0.4677 - val_loss: 1.6761 - val_categorical_crossentropy: 1.7481 - val_accuracy: 0.3900
Epoch 4/10
80/80 [==============================] - 21s 260ms/step - loss: 1.2694 - categorical_crossentropy: 1.4311 - accuracy: 0.5454 - val_loss: 1.5477 - val_categorical_crossentropy: 1.4664 - val_accuracy: 0.5400
Epoch 5/10
80/80 [==============================] - 21s 261ms/step - loss: 1.1483 - categorical_crossentropy: 1.2986 - accuracy: 0.5